In [ ]:
import pandas as pd

path = '/Users/brunobarbieri/Library/CloudStorage/OneDrive-UniversityofPisa/TA_Project/data/'
df = pd.read_csv(path + 'lab_lem_merge.csv')

In [ ]:
df = df.drop(columns= ['Unnamed: 0'])
df

,id,title,artist,year,views,features,is_country,is_pop,is_rap,is_rb,is_rock,stanza_number,is_chorus,lemmatized_stanzas,label
0,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,0,False,"['she', 'get', 'a', 'broke', 'down', 'el', 'ca...",anger
1,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,1,True,"['something', 'in', 'the', 'water', 'something...",anger
2,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,2,False,"['she', 'do', 'her', 'makeup', 'and', 'hair', ...",anticipation
3,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,4,False,"['my', 'hoosi', 'girl', 'be', 'so', 'fine', 's...",fear
4,0,Something in the Water,Pokey LaFarge,2015,10902,{''},True,False,False,False,False,5,True,"['something', 'in', 'the', 'water', 'something...",fear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57955,11010,Lay It in My Hand,The Real Group,2009,71,{''},False,True,False,False,False,6,False,"['come', 'and', 'release', 'your', 'confidence...",trust
57956,11011,Stop Calling,Lina Morgana,2008,74,{''},False,True,False,False,False,0,False,"['when', 'the', 'sun', 'kill', 'the', 'moon', ...",trust
57957,11011,Stop Calling,Lina Morgana,2008,74,{''},False,True,False,False,False,1,False,"['and', 'I', 'do', 'not', 'care', 'anymore', '...",fear
57958,11011,Stop Calling,Lina Morgana,2008,74,{''},False,True,False,False,False,2,True,"['stop', 'call', 'I', 'have', 'stop', 'care', ...",fear


In [5]:
df.to_csv(path + 'lab_lem_merged.csv', index= False)

# 1D Convnet

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Initialize the tokenizer
vocab_size = 20000
tokenizer = Tokenizer(num_words= vocab_size, oov_token="<UNK>")
tokenizer.fit_on_texts(df['lemmatized_stanzas'])

sequences = tokenizer.texts_to_sequences(df['lemmatized_stanzas'])

# Save the tokenizer for future preprocessing
import joblib

tokenizer_path = '/Users/brunobarbieri/Library/CloudStorage/OneDrive-UniversityofPisa/TA_Project/tokenizers/'

joblib.dump(tokenizer, tokenizer_path + 'basic_tokenizer.pkl')

2024-12-03 16:53:28.382481: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[[33,
  22,
  8,
  2514,
  52,
  2731,
  9580,
  13,
  4,
  569,
  1237,
  38,
  18,
  1001,
  118,
  1147,
  143,
  119,
  13,
  8,
  1273,
  13683,
  7076,
  7,
  356,
  2296,
  33,
  12,
  9,
  115,
  8,
  2604,
  367,
  13,
  4,
  681,
  15,
  4,
  1281,
  7,
  4,
  251,
  33,
  144,
  8,
  271,
  13,
  11,
  671,
  282,
  2,
  859,
  28,
  2,
  27,
  33,
  30,
  4,
  195,
  152,
  27,
  33,
  30,
  4,
  195],
 [145,
  13,
  4,
  318,
  145,
  13,
  4,
  13684,
  13,
  4,
  318,
  145,
  13,
  4,
  13685,
  137,
  137,
  137,
  40,
  49,
  33,
  276,
  2,
  12,
  9,
  26,
  54,
  48,
  346,
  31,
  33,
  155,
  45,
  281,
  3,
  145,
  13,
  4,
  318],
 [33,
  12,
  118,
  1904,
  7,
  406,
  6,
  1850,
  7722,
  1403,
  13,
  118,
  2930,
  33,
  279,
  11188,
  1653,
  24,
  3007,
  7,
  1456,
  7,
  453,
  2,
  117,
  692,
  33,
  1419,
  33,
  540,
  7,
  33,
  296,
  2,
  28,
  2,
  12,
  9,
  115,
  4,
  59,
  33,
  642,
  2,
  33,
  209,
  550,
  430,
  6,
  11,
  471,
  2,


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_seq_length = 150  # Adjust based on your average stanza length
padded_sequences = pad_sequences(
    sequences, maxlen=max_seq_length,
    padding='post', truncating='post'
)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

# Define the parameters for the stub implementation
embedding_dim = 128  # Dimensions for the embedding layer
num_classes = 8

# Create a simple 1D ConvNet model
def create_1d_convnet(vocab_size, embedding_dim, max_length, num_classes):
    model = Sequential([
        Embedding(
            input_dim=vocab_size, output_dim=embedding_dim,
            input_length=max_length, name='embedding_layer'
        ),
        Conv1D(
            filters=64, kernel_size=3,
            activation='relu', name='conv1d_layer'
        ),
        GlobalMaxPooling1D(name='global_max_pooling'),
        Dense(64, activation='relu', name='dense_layer'),
        Dense(
            num_classes, activation='softmax',
            name='output_layer'
        )
    ])
    return model

2024-12-03 15:23:59.366023: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Instantiate the model
model = create_1d_convnet(vocab_size, embedding_dim, max_length, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_layer (Embedding)     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_layer (Conv1D)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling              │ ?                      │   0 (unbuilt) │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_layer (Dense)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# trains, saves model
def train_and_save_model(model, X_train, y_train, X_val, y_val,
                         model_path, epochs=10, batch_size=32):
    # Train the model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size
    )
    # Save the trained model
    model.save(model_path)
    print(f"Trained model saved to {model_path}")
    return history